In [1]:
import os
from pathlib import Path
import pandas as pd
import numpy as np

In [2]:
PROJECT_ROOT = Path(os.getcwd()).resolve().parents[0]

In [36]:
def get_year_month(row):
    date_str = row["date"]
    try:
        parts = date_str.split('.')
        year = int(parts[1])
        month = int(parts[0])
        row["mois"] = month
        row["année"] = year
        del row["date"]
        return row
    except:
        print("error")

# 13 - 17

In [20]:
df_13 = pd.read_excel(PROJECT_ROOT / "data/raw/Données GC 2013 -2017.xlsx", header=None)

In [23]:
df_13.head()

,0,1,2,3,4,5,6,7,8,9,...,176,177,178,179,180,181,182,183,184,185
0,NaN,NaN,NaN,NaN,NaN,NaN,Ventes ONEE en KWh,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Catégorie,Contrat,Partenaire,NaN,Activité,Secteur,2013,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,Contrat,NaN,NaN,1,2.0,3.0,4.0,...,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0
145,Distributeurs,888950,Distributeurs 8,A,NaN,NaN,3557388,3139542.0,3557490.0,3241386.0,...,4973259.0,4636291.0,4593394.0,4935703.0,5822420.0,6437199.0,6240114.0,5419130.0,5017715.0,5482143.0
146,Distributeurs,217006,Distributeurs 10,A,NaN,NaN,2418637,2060201.0,2316780.0,2264971.0,...,0.0,0.0,2.0,0.0,0.0,2.0,0.0,56.0,1.0,0.0


In [22]:
df_13 = df_13[df_13[0] != "C. Directs"]

In [24]:
df_13.drop(columns = [0, 3, 4, 5], inplace=True)

In [25]:
for i in range(len(df_13.iloc[0])):
    value = df_13.iloc[0, i]
    if pd.notna(value):
        print(i, value)

2 Ventes ONEE en KWh
62 Achat en KWh auprés  des producteurs privés 
122 Consommation en KWh 


In [26]:
df_13.head()

,1,2,6,7,8,9,10,11,12,13,...,176,177,178,179,180,181,182,183,184,185
0,NaN,NaN,Ventes ONEE en KWh,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Contrat,Partenaire,2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,1,2.0,3.0,4.0,5.0,6.0,7.0,8.0,...,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0
145,888950,Distributeurs 8,3557388,3139542.0,3557490.0,3241386.0,3428610.0,3896682.0,3665928.0,3727644.0,...,4973259.0,4636291.0,4593394.0,4935703.0,5822420.0,6437199.0,6240114.0,5419130.0,5017715.0,5482143.0
146,217006,Distributeurs 10,2418637,2060201.0,2316780.0,2264971.0,2337319.0,2471647.0,2870907.0,3048401.0,...,0.0,0.0,2.0,0.0,0.0,2.0,0.0,56.0,1.0,0.0


In [27]:
new_cols_13 = ['contrat', 'partenaire']
new_cols_13 += [f"{j}.{i}" for i in range(2013, 2018) for j in range(1,13)]

In [28]:
df_13 = df_13.iloc[3:]
df_13.dropna(subset=[1,2], inplace=True)

## ventes

In [41]:
df_13_ventes = df_13.iloc[:, :64]

In [42]:
df_13_ventes.shape

(142, 64)

In [43]:
df_13_ventes.columns = new_cols_13

In [44]:
df_13_ventes = df_13_ventes.melt(
    id_vars=['contrat', 'partenaire', 'activité', 'secteur'],
    var_name='date',
    value_name='ventes'  
)

In [45]:
df_13_ventes = df_13_ventes.apply(get_year_month, axis=1)

## PV

In [160]:
df_13_pv = pd.concat([df_13.iloc[:, :4],df_13.iloc[:, 64:124]], axis=1)

In [187]:
df_13_pv.shape

(136, 64)

## Consomation

In [29]:
df_13_consomation = pd.concat([df_13.iloc[:, :2],df_13.iloc[:, 122:]], axis=1)

In [30]:
df_13_consomation.shape

(52, 62)

In [32]:
df_13_consomation.columns = new_cols_13

In [34]:
df_13_consomation = df_13_consomation.melt(
    id_vars=['contrat', 'partenaire'],
    var_name='date',
    value_name='consomation'  
)

In [35]:
df_13_consomation

,contrat,partenaire,date,consomation
0,888950,Distributeurs 8,1.2013,3557388
1,217006,Distributeurs 10,1.2013,2418637
2,217008,Distributeurs 10,1.2013,24430165
3,217009,Distributeurs 10,1.2013,15364686
4,217012,Distributeurs 5,1.2013,20662631
...,...,...,...,...
3115,4925969,Distributeurs 6,12.2017,11638608.0
3116,5084094,Distributeurs 11,12.2017,17186664.0
3117,5084143,Distributeurs 12,12.2017,35877303.0
3118,5681944,Distributeurs 12,12.2017,4853783.0


In [37]:
df_13_consomation = df_13_consomation.apply(get_year_month, axis=1)

In [38]:
df_13_consomation

,contrat,partenaire,consomation,mois,année
0,888950,Distributeurs 8,3557388.0,1,2013
1,217006,Distributeurs 10,2418637.0,1,2013
2,217008,Distributeurs 10,24430165.0,1,2013
3,217009,Distributeurs 10,15364686.0,1,2013
4,217012,Distributeurs 5,20662631.0,1,2013
...,...,...,...,...,...
3115,4925969,Distributeurs 6,11638608.0,12,2017
3116,5084094,Distributeurs 11,17186664.0,12,2017
3117,5084143,Distributeurs 12,35877303.0,12,2017
3118,5681944,Distributeurs 12,4853783.0,12,2017


In [41]:
df_13_consomation.rename(columns = {"mois":"month", "année":"year", "consomation":"consumption_value", "partenaire":"distributeur"}, inplace = True)

In [45]:
df_13_consomation.head()

,contrat,distributeur,consumption_value,month,year
0,888950,Distributeurs 8,3557388.0,1,2013
1,217006,Distributeurs 10,2418637.0,1,2013
2,217008,Distributeurs 10,24430165.0,1,2013
3,217009,Distributeurs 10,15364686.0,1,2013
4,217012,Distributeurs 5,20662631.0,1,2013


In [52]:
distributeurs = {
    "Distributeurs 1": "LYDEC CASABLANCA",
    "Distributeurs 2": "RADEEL LARACHE",
    "Distributeurs 3": "RADEEMA MARRAKECH",
    "Distributeurs 4": "RADEES SAFI",
    "Distributeurs 5": "RADEM MEKNES",
    "Distributeurs 6": "RAK KENITRA",
    "Distributeurs 7": "REDAL RABAT",
    "Distributeurs 8": "Tanger Med Utilities",
    "Distributeurs 9": "AMENDIS TETOUAN",
    "Distributeurs 10": "RADEEF FES",
    "Distributeurs 11": "RADEEJ EL JADIDA",
    "Distributeurs 12": "AMENDIS TANGER"
}

In [58]:
df_13_consomation["distributeur"] = df_13_consomation["distributeur"].apply(lambda x: distributeurs[x])

In [64]:
distributeur_regions = {
    "AMENDIS TANGER": "Tanger-Tétouan-Al Hoceïma",
    "AMENDIS TETOUAN": "Tanger-Tétouan-Al Hoceïma",
    "LYDEC CASABLANCA": "Casablanca-Settat",
    "RADEEF FES": "Fès-Meknès",
    "RADEEJ EL JADIDA": "Casablanca-Settat",
    "RADEEL LARACHE": "Tanger-Tétouan-Al Hoceïma",
    "RADEEMA MARRAKECH": "Marrakech-Safi",
    "RADEES SAFI": "Marrakech-Safi",
    "RADEM MEKNES": "Fès-Meknès",
    "RAK KENITRA": "Rabat-Salé-Kénitra",
    "REDAL RABAT": "Rabat-Salé-Kénitra",
    "Tanger Med Utilities": "Tanger-Tétouan-Al Hoceïma"
}


In [66]:
df_13_consomation["region"] = df_13_consomation["distributeur"].apply(lambda x: distributeur_regions[x])

In [68]:
df_13_consomation.to_csv(PROJECT_ROOT / "data/distributeur_2013_2017.csv", index = False)

In [3]:
df = pd.read_csv(PROJECT_ROOT / "data/distributeur_2013_2017.csv")

In [4]:
df

,contrat,distributeur,consumption_value,month,year,region
0,888950,Tanger Med Utilities,3557388.0,1,2013,Tanger-Tétouan-Al Hoceïma
1,217006,RADEEF FES,2418637.0,1,2013,Fès-Meknès
2,217008,RADEEF FES,24430165.0,1,2013,Fès-Meknès
3,217009,RADEEF FES,15364686.0,1,2013,Fès-Meknès
4,217012,RADEM MEKNES,20662631.0,1,2013,Fès-Meknès
...,...,...,...,...,...,...
3115,4925969,RAK KENITRA,11638608.0,12,2017,Rabat-Salé-Kénitra
3116,5084094,RADEEJ EL JADIDA,17186664.0,12,2017,Casablanca-Settat
3117,5084143,AMENDIS TANGER,35877303.0,12,2017,Tanger-Tétouan-Al Hoceïma
3118,5681944,AMENDIS TANGER,4853783.0,12,2017,Tanger-Tétouan-Al Hoceïma


In [8]:
import sqlite3
db_orig = sqlite3.connect(PROJECT_ROOT / "data/ONEE_Distributeurs_consumption.db")

In [9]:
table_name = "consumption"


# 1. Load existing data from the database
df_existing = pd.read_sql(f"SELECT * FROM {table_name}", db_orig)

In [10]:
df_existing

,distributeur,region,year,month,poste_horaire,use,consumption_value
0,AMENDIS TANGER,Tanger-Tétouan-Al Hoceïma,2018,1,ZCONHC,creuse,7768.040
1,AMENDIS TANGER,Tanger-Tétouan-Al Hoceïma,2018,1,ZCONHL,pleine,8987.200
2,AMENDIS TANGER,Tanger-Tétouan-Al Hoceïma,2018,1,ZCONHP,pointe,4597.783
3,AMENDIS TANGER,Tanger-Tétouan-Al Hoceïma,2018,1,ZCONHC,creuse,11207.366
4,AMENDIS TANGER,Tanger-Tétouan-Al Hoceïma,2018,1,ZCONHL,pleine,15571.653
...,...,...,...,...,...,...,...
11191,REDAL RABAT,Rabat-Salé-Kénitra,2023,12,ZCONHL,pleine,11413.727
11192,REDAL RABAT,Rabat-Salé-Kénitra,2023,12,ZCONHP,pointe,6229.648
11193,Tanger Med Utilities,Tanger-Tétouan-Al Hoceïma,2023,12,ZCONHC,creuse,4772.986
11194,Tanger Med Utilities,Tanger-Tétouan-Al Hoceïma,2023,12,ZCONHL,pleine,4850.682


In [15]:
df.drop(columns = ["contrat"], inplace=True)

In [17]:
# 2. Simulate concatenation (no write yet)
df_combined = pd.concat([df_existing, df], ignore_index=True).sort_values(by=["year", "month"])

In [19]:

df_combined.to_sql(table_name, db_orig, if_exists="replace", index=False)

db_orig.close()